<a href="https://colab.research.google.com/github/un21988/CE888-7-project-2/blob/main/Offensive_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Offensive Classification using Roberta** 





In [2]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [3]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.4MB/s 
     |████████████████████████████████| 901kB 55.6MB/s 
     |████████████████████████████████| 3.3MB 57.4MB/s 


In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='offensive'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)


In [5]:
# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[0] for row in csvreader if len(row) > 0]
labels

['0', '1']

In [6]:
import pandas as pd
import urllib.request 
import requests

test_text_url = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt"
val_text_url= "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_text.txt"
val_lable_url= "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt"
#df = open(url,"r")
#print(df)


r = requests.get(test_text_url, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r1 = requests.get(val_text_url, allow_redirects=True)
open('val_text.txt', 'wb').write(r1.content)


r2 = requests.get(val_lable_url, allow_redirects=True)
open('test_lable.txt', 'wb').write(r2.content)






1720

In [7]:
stream=open("test_text.txt")
tweets=stream.readlines()
stream.close()
print(tweets[1])

@user @user @user I got in a pretty deep debate with my friend and she told me that latinos for Trump and blacks for Trump were paid supporters 😂 then I said you mean antifa are paid domestic terrorist and she said No they are  anti-fascist then I said they are the fascist are you kidding me?! 



In [8]:
stream1=open("test_lable.txt")
test_lable=stream1.readlines()
stream1.close()
#print(test_lable)
test_data=[]
print(type(test_lable[1]))
for i in range(0,len(test_lable)):
  temp_data=test_lable[i]
  test_data.append(int(temp_data))

#print(test_data)

<class 'str'>


In [9]:
stream2=open("val_text.txt")
val_text=stream2.readlines()
stream2.close()
print(val_text[5])

@user @user @user We could help if you are London based 😊 



In [10]:
tweet_preprocess=[]

for i in range(0,len(tweets)):
  tweet_preprocess.append(preprocess(tweets[i]))


print(tweet_preprocess[0])

#ibelieveblaseyford is liar she is fat ugly libreal #snowflake she sold her  herself to get some cash !! From dems and Iran  ! Why she spoke after  #JohnKerryIranMeeting ? 



In [14]:
result=[]
for i in  range(0,len(tweet_preprocess)):
  result.append(model(tweet_preprocess[i]))

 
#print(result)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-offensive.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-offensive.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the mode

In [19]:

final_result =[]
predicted_data=[] # The result we are getting by running the model
for i in  range(0,len(result)):
  final_result.append(min(result[i]))



for i in range(0,len(final_result)):
  extract_data=final_result[i][3]
  predicted_data.append(int(extract_data))
  extract_data= 0

  #print("Predicted value: " , predicted_data[i])
  #print(type(predicted_data[i]))
  #print ("Test value: " , test_data[i])

In [20]:
#--------------------------To check the output which is able to use or not -----------

for i in range(0,len(test_data)):
  print(test_data[i])
  if i== 10:
    break


1
0
0
0
0
0
0
0
1
0
1


In [21]:
#--------------------------To check the output which is able to use or not -----------

for i in range(0,len(predicted_data)):
  print(predicted_data[i])
  if i== 10:
    break


1
1
0
0
0
0
0
0
1
0
1


In [22]:
#-------------------Saving the results of the model in  file for the evaluation perpose  ------------
with open('offensive_classification_result.txt', 'w') as f:
    for item in predicted_data:
        f.write("%s\n" % item)

In [23]:
#------------checking the acuuracy ----------------
def getAccuracy(test_data,predicted_data):
   n = len(test_data) 
   correct = 0
   for x in range(n):
       if test_data[x] == predicted_data[x]:
           correct += 1
   return (correct/n) * 100.0




accuracy = getAccuracy(test_data ,predicted_data)
print("we got the accuracy by ", round(accuracy),"%")

we got the accuracy by  86 %


In [12]:
def model(tweets):

 model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

 text = tweets
 encoded_input = tokenizer(text, return_tensors='tf')
 output = model(encoded_input)
 scores = output[0][0].numpy()
 scores = softmax(scores)

 temp_list=[]
 ranking = np.argsort(scores)
 ranking = ranking[::-1]
 for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    temp_list.append(f"{i+1}) {l} {np.round(float(s), 4)}")
 return temp_list